In [ ]:
import zipfile, os

with zipfile.ZipFile("../input/dogs-vs-cats/train.zip","r") as z:
    z.extractall(".")

In [ ]:
img_width, img_height = 150, 150
input_shape = (img_width, img_height, 3)
batch_size = 15
nb_train_samples = 20000
nb_validation_samples = 5000

nb_images = 25000
nb_classes = 2
nb_class_images = nb_images // nb_classes

val_data_portion = 0.2

In [ ]:
def create_directory(dir_name):
    if os.path.exists(dir_name):
        shutil.rmtree(dir_name)
    os.makedirs(dir_name)
    os.makedirs(os.path.join(dir_name, "cats"))
    os.makedirs(os.path.join(dir_name, "dogs"))

In [ ]:
data_dir = '/kaggle/working/train'
train_dir = '/kaggle/working/set/train' 
val_dir = '/kaggle/working/set/val' 

In [ ]:
create_directory(train_dir)
create_directory(val_dir)

In [ ]:
import shutil

def copy_images(start_index, end_index, source_dir, dest_dir):
    for i in range(start_index, end_index):
        shutil.copy2(os.path.join(source_dir, "cat." + str(i) + ".jpg"), 
                     os.path.join(dest_dir, "cats"))
        shutil.copy2(os.path.join(source_dir, "dog." + str(i) + ".jpg"),
                     os.path.join(dest_dir, "dogs"))

In [ ]:
start_val_data_idx = int(nb_class_images * (1-val_data_portion))


copy_images(0, start_val_data_idx, data_dir, train_dir)
copy_images(start_val_data_idx, 12500, data_dir, val_dir)

In [ ]:
print('Cats in train set: ', len(os.listdir(train_dir + '/cats')))
print('Dogs in train set: ', len(os.listdir(train_dir + '/dogs')), '\n')

print('Cats in validation set: ', len(os.listdir(val_dir + '/cats')))
print('Cats in validation set: ', len(os.listdir(val_dir + '/dogs')), '\n')

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.python.keras.preprocessing import image

# Cats

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    image_file_name = train_dir + '/cats/' + os.listdir(train_dir+"/cats")[i] 
    img = image.load_img(image_file_name, target_size=(150, 150))
    plt.imshow(img)
plt.show()

# Dogs

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    image_file_name = train_dir + '/dogs/' + os.listdir(train_dir+"/dogs")[i] 
    img = image.load_img(image_file_name, target_size=(150, 150))
    plt.imshow(img)
plt.show()

# Data Augmentation

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1
)

In [ ]:
image_file_name = train_dir + '/cats/cat.11.jpg'
img = image.load_img(image_file_name, target_size=(150, 150))
plt.imshow(img)

In [ ]:
x = image.img_to_array(img)
x = x.reshape((1,) + x.shape)
i = 0
for batch in train_datagen.flow(x, batch_size=1):
    plt.figure(i)
    imgplot = plt.imshow(image.array_to_img(batch[0]))
    i += 1
    if i == 5:
        break
plt.show()

# Creating Model using transfer learning with ResNet_V2

In [ ]:
from tensorflow import keras
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.applications import resnet_v2

In [ ]:
transfer_net = resnet_v2.ResNet50V2(weights ='imagenet', include_top = False, input_shape = input_shape)
transfer_net.trainable = False

In [ ]:
model = Sequential()
model.add(transfer_net)
model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = Adam(lr=0.00001), loss ='binary_crossentropy', metrics = ['accuracy'])

In [ ]:
datagen = ImageDataGenerator(rescale = 1./255)
train_gen = train_datagen.flow_from_directory(train_dir, target_size = (img_width, img_height), batch_size = batch_size, class_mode ='binary')
val_gen = datagen.flow_from_directory(val_dir, target_size = (img_width, img_height), batch_size = batch_size, class_mode ='binary')

In [ ]:
history = model.fit(train_gen, 
                    steps_per_epoch = nb_train_samples // batch_size, 
                    epochs = 10, 
                    validation_data = val_gen, 
                    validation_steps = nb_validation_samples // batch_size)

In [ ]:
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12))
ax1.plot(history.history['loss'], color='b', label="Train_loss")
ax1.plot(history.history['val_loss'], color='r', label="Val_loss")
ax1.set_xticks(np.arange(1, 10, 1))
ax1.set_yticks(np.arange(0, 1, 0.1))

ax2.plot(history.history['accuracy'], color='b', label="Train_accuracy")
ax2.plot(history.history['val_accuracy'], color='r',label="Val_accuracy")
ax2.set_xticks(np.arange(1, 10, 1))

legend = plt.legend(loc='best', shadow=True)
plt.tight_layout()
plt.show()

# Model save

In [ ]:
model.save_weights("model.h5")

# Prepare test images

In [ ]:
import zipfile, os

with zipfile.ZipFile("../input/dogs-vs-cats/test1.zip","r") as z:
    z.extractall(".")

In [ ]:
test_filenames = os.listdir("/kaggle/working/test1")
test_df = pd.DataFrame({
    'filename': test_filenames
})
nb_samples = test_df.shape[0]

In [ ]:
batch_size = 10

test_gen = ImageDataGenerator(rescale=1./255)
test_generator = test_gen.flow_from_dataframe(
    test_df, 
    "/kaggle/working/test1", 
    x_col='filename',
    y_col=None,
    class_mode=None,
    target_size=(150, 150),
    batch_size=batch_size,
    shuffle=False
)

# Predict

In [ ]:
predict = model.predict_generator(test_generator, steps=nb_samples//batch_size)

In [ ]:
predict.shape

In [ ]:
test_df.drop(['category'], axis=1, inplace=True)

In [ ]:
category = np.rint(predict)
test_df['label'] = category

In [ ]:
test_df

# Save result

In [ ]:
submission = test_df.copy()
submission['id'] = submission['filename'].str.split('.').str[0]
submission.drop(['filename'], axis=1, inplace=True)
submission.to_csv('submission.csv', index=False)